In [2]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from pathlib import Path
import scipy.sparse as sparse
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k
from lightfm.data import Dataset

In [3]:
df_sub = pd.read_csv(Path.cwd() / 'data' / 'sample_submission.csv')
df_train = pd.read_csv(Path.cwd() / 'data' / 'train.csv')
df_test = pd.read_csv(Path.cwd() / 'data' / 'test.csv')

/home/alex/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/home/alex/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
NUM_THREADS = 8 #число потоков
NUM_COMPONENTS = 60 #число параметров вектора 
NUM_EPOCHS = 25 #число эпох обучения

In [5]:
df_train.drop_duplicates(inplace=True)

In [6]:
df_train_train, df_train_test = train_test_split(df_train, random_state=32, shuffle=True)

In [7]:
dataset = Dataset()

In [8]:
#Создаем датасет с размерностью макс. юзер на макс. итем.
dataset.fit(
            users=range(df_train.userid.max()+1),
            items=range(df_train.itemid.max()+1) #,
          #  user_features=['overall:1', 'overall:2', 'overall:3', 'overall:4', 'overall:5'],
          #  item_features=[]
)

In [9]:
dataset.model_dimensions()

(127496, 41320)

In [10]:
df_train_train[:1].apply(lambda x: (x.userid, {'overall': x.overall}), axis=1)

728049    (3215, {'overall': 5.0})
dtype: object

Создаем матрицу взаимодействий на всем train. Пытаемся повторить baseline без dataset()

In [11]:
(interactions, weights) = dataset.build_interactions(
    [(x[11], x[12], int(x[13])) for x in df_train.values])

In [12]:
user_id_map, user_feature_map, item_id_map, item_feature_map = dataset.mapping()

In [13]:
item_id_map[1333]

1333

In [14]:
ratings_coo = sparse.coo_matrix((df_train['rating'].astype(int),
                                 (df_train['userid'],
                                  df_train['itemid'])))

In [36]:
(interactions.data == ratings_coo.data)[1:30]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False, False,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [38]:
interactions.data[13]

1

In [39]:
ratings_coo.data[13]

0

In [16]:
ratings_coo.row

array([102179,   3625,  39495, ...,     88,  41540,    466], dtype=int32)

In [17]:
ratings_coo.col

array([37138, 17322,  5600, ...,  3251, 22208,  2931], dtype=int32)

In [18]:
interactions.col

array([37138, 17322,  5600, ...,  3251, 22208,  2931], dtype=int32)

In [19]:
interactions.data.max()

1

In [20]:
ratings_coo.data.max()

1

In [23]:
ratings_coo != interactions

<127496x41320 sparse matrix of type '<class 'numpy.bool_'>'
	with 124337 stored elements in Compressed Sparse Row format>

In [91]:
model = LightFM(learning_rate=0.1, loss='logistic',
                no_components=NUM_COMPONENTS)

In [92]:
model.fit(interactions, 
            sample_weight=weights,
            epochs=NUM_EPOCHS, 
            num_threads=NUM_THREADS)

In [62]:
model.get_user_representations()[0][0:5]

array([0.44872344, 1.3521495 , 1.5934019 , 0.79721636, 1.0134473 ],
      dtype=float32)

In [15]:
from lightfm.evaluation import auc_score
train_auc = auc_score(model,
                      interactions
                     ).mean()
print('Hybrid training set AUC: %s' % train_auc)

Hybrid training set AUC: 0.8193337


In [93]:
preds = model.predict(df_train_test.userid.values,
                      df_train_test.itemid.values,
                     )

In [95]:
roc_auc_score(df_train_test.rating, preds)

0.5908378131107098

In [45]:
preds[0]

4.026906490325928

In [83]:
df_train_test.columns

Index(['overall', 'verified', 'reviewTime', 'asin', 'reviewerName',
       'reviewText', 'summary', 'unixReviewTime', 'vote', 'style', 'image',
       'userid', 'itemid', 'rating'],
      dtype='object')

In [121]:
def cold_start(row, user_id_map, item_id_map, model):
    if row.userid in user_id_map and row.itemid in item_id_map:
        return model.predict(user_id_map[row.userid], [item_id_map[row.itemid]])[0]
    else:
        print(row.userid, row.itemid)
        return 0

In [59]:
model.predict(1, [0, 1, 2, 3])

array([2.87358022, 3.31695056, 2.79585004, 4.53327513])

In [122]:
pred_set = df_test[0:1000].apply(cold_start, axis=1, user_id_map=user_id_map, 
                    item_id_map=item_id_map, model=model)

13457 18145
85892 37102
56698 28650
124621 33986
110673 12113
37076 13307


In [59]:
pred_set = model.predict(df_test.userid.values,
                      df_test.itemid.values)

In [60]:
pred_set.min(), pred_set.max()

(-0.0002898226666729897, 11.438776016235352)

In [24]:
normalized_preds = (pred_set - pred_set.min())/(pred_set - pred_set.min()).max()

In [25]:
normalized_preds.min(), normalized_preds.max()

(0.0, 1.0)

In [26]:
df_sub['rating']= normalized_preds

In [27]:
df_sub.to_csv('submission_dataset_log.csv', index=False)